<a href="https://colab.research.google.com/github/carolineferguson/trabalho_dataops/blob/main/trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.1 MB/s e

Autenticação do banco no Cloud SQL

In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "bright-task-382122" #@param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Updated IAM policy for project [bright-task-382122].
bindings:
- members:
  - user:caroline.ferguson86@gmail.com
  role: roles/cloudsql.client
- members:
  - serviceAccount:service-667361598200@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:667361598200@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - serviceAccount:service-667361598200@gcp-sa-migcenter.iam.gserviceaccount.com
  role: roles/migrationcenter.serviceAgent
- members:
  - serviceAccount:service-667361598200@gcp-sa-networkmanagement.iam.gserviceaccount.com
  role: roles/networkmanagement.serviceAgent
- members:
  - user:caroline.ferguson86@gmail.com
  role: roles/owner
- members:
  - serviceAccount:667361598200-compute@developer.gserviceaccount.com
  - serviceAccount:ativiade-dataops@bright-task-382122.iam.gserviceaccount.com
  role: roles/storage.admin
- members:
  - serviceAccount:ativiade-dataops@bright-task-382122.iam.gserviceaccount.com
  role

In [ ]:
!gcloud services enable sqladmin.googleapis.com

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name for your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1" #@param {type:"string"}
instance_name = "ativiadde-dataops" #@param {type:"string"}

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("MYSQL"):
  print("Found existing MySQL Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  password = input("Please provide a password to be used for database 'root' user: ")
  !gcloud sql instances create {instance_name} --database-version=MYSQL_8_0 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql_iam_authentication=On

Found existing MySQL Cloud SQL Instance!


In [ ]:
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "usuario"
DB_PASS = "root"
DB_NAME = "data"

Your instance connection name is: bright-task-382122:us-central1:ativiadde-dataops


In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

Scraping

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
 
def scrape_this(uri="/pages/forms/"):
  page = requests.get("https://scrapethissite.com" + uri)
  soup = BeautifulSoup(page.text, "html.parser")

  div = soup.find(id="hockey")  
  table = div.find("table")

  data_rows = table.find_all("tr", attrs={"class": "team"})
  parsed_data = list()
  stat_keys = [col.attrs["class"][0] for col in data_rows[0].find_all("td")]

  for row in data_rows:
    tmp_data = dict()
    for attr in stat_keys:
      attr_val = row.find(attrs={"class": attr}).text
      tmp_data[attr] = re.sub(r"^\s+|\s+$", "", attr_val)
    parsed_data.append(tmp_data)

  data_df = pd.DataFrame(parsed_data)
  return data_df

page = requests.get("https://scrapethissite.com/pages/forms/")
soup = BeautifulSoup(page.text, "html.parser")
pagination = soup.find("ul", attrs={"class": "pagination"})
link_elms = pagination.find_all("li")
links = [link_elm.find("a").attrs["href"] for link_elm in link_elms]
links = set(links)

temp_dfs = list()
for link in links:
  tmp_df = scrape_this(uri=link)
  temp_dfs.append(tmp_df)
hockey_team_df = pd.concat(temp_dfs, axis=0).reset_index()
hockey_team_df.sort_values(["year", "name"], inplace=True)

Através do dataframe com conector com o banco, colocamos este dataframe na tabela

In [ ]:
# connect to connection pool
with pool.connect() as db_conn:
  hockey_team_df.to_sql("hockey2", db_conn)

  # commit transactions
  db_conn.commit()

  # query and fetch ratings table
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM hockey2")).fetchall()

  # show results
  for row in results:
    print(row)

(282, 0, 'Boston Bruins', '1990', '44', '24', '', '0.55', '299', '264', '35')
(283, 1, 'Buffalo Sabres', '1990', '31', '30', '', '0.388', '292', '278', '14')
(284, 2, 'Calgary Flames', '1990', '46', '26', '', '0.575', '344', '263', '81')
(285, 3, 'Chicago Blackhawks', '1990', '49', '23', '', '0.613', '284', '211', '73')
(286, 4, 'Detroit Red Wings', '1990', '34', '38', '', '0.425', '273', '298', '-25')
(287, 5, 'Edmonton Oilers', '1990', '37', '37', '', '0.463', '272', '272', '0')
(288, 6, 'Hartford Whalers', '1990', '31', '38', '', '0.388', '238', '276', '-38')
(289, 7, 'Los Angeles Kings', '1990', '46', '24', '', '0.575', '340', '254', '86')
(290, 8, 'Minnesota North Stars', '1990', '27', '39', '', '0.338', '256', '266', '-10')
(291, 9, 'Montreal Canadiens', '1990', '39', '30', '', '0.487', '273', '249', '24')
(292, 10, 'New Jersey Devils', '1990', '32', '33', '', '0.4', '272', '264', '8')
(293, 11, 'New York Islanders', '1990', '25', '45', '', '0.312', '223', '290', '-67')
(294, 12,

In [ ]:
import pandas as pd
with pool.connect() as db_conn:
  df = pd.read_sql_table("hockey2", con=db_conn)

Ingestão através de uma pipeline no cluster da ElasticSearch

In [ ]:
use_these_keys = ['index','name','year','wins','losses','ot-losses','pct','gf','ga','diff']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

In [ ]:
from elasticsearch import Elasticsearch, exceptions
from elasticsearch import helpers
es_client = Elasticsearch(cloud_id= 'pipeline_atividade:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRjYTdhOTZmMzhhN2Y0MjBiYmE0ZDA2NmFlMzEyMTY1ZiRlOTVmZjNkMmFjNmE0YmNlYTRjNmY5OTgzMWQ1OTA5ZA==',basic_auth=("elastic","PIdv8msR0uHYpFMAFZdaBTb5"))
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'search-id',
                "_id" : f"{index}",
                "_source": filterKeys(document),
        }
try:
  helpers.bulk(es_client, doc_generator(df))
except exceptions.NotFoundError:
    pass

Conectando ao github com o workflow na pasta do drive, o git actions deixará a pipeline automatizada